In [1]:
print('hello world')

hello world


In [ ]:
import nltk
from nltk.tokenize import word_tokenize 





In [ ]:
pip install torchtext

In [ ]:
text = 'eel aik aisi shakhsiyat ko kaha jata hai ke jo dosray apne Sarif ki janib se ya uski baabat guftagu kere is mazmoon mein yeh guftagu qanoon se mutaliq'

print(word_tokenize(text))

In [ ]:
with open('./training_data/Roman-Urdu.txt', 'r', encoding='utf-8') as f:
    romanUrduLines = f.readlines()[:200]

with open('./training_data/Urdu.txt', 'r', encoding='utf-8') as f:
    urduLines = f.readlines()[:200]


    

In [ ]:
print(linesSrc[0:25])

In [ ]:
print('hello world')

In [ ]:
srcTokens = []
trgTokens = []

pair = (0,0)
with open ('./training_data/Roman-Urdu.txt') as sourceFile, open ('./training_data/Urdu.txt') as targetFile:
    srcTokens = []
    trgTokens = []
    for i in range( len(sourceFile.)):
        srcLine = sourceFile[i]
        srcLine.replace('/n', ' ')
        srcTokens = srcLine.split(' ')

        trgLine = targetFile[i]
        trgLine.replace('/n', ' ')
        trgTokens = line.split(' ')

        pair = (srcTokens[3], trgToken[3])

    
        



print(srcTokens[0:25], trgTokens[0:25])


In [ ]:
source = open('./training_data/Roman-Urdu.txt')
doc_source = source.read()



In [ ]:
import torch

# Class to load data

class Dataset(torch.utils.data.Dataset):

    def __init__(self) -> None:
        self.source = open('./training_data/Roman-Urdu.txt', 'r', encoding='utf-8').read().split('\n')
        self.target = open('./training_data/Urdu.txt', 'r', encoding='utf-8').read().split('\n')


    def __getitem__(self, idx):
        source_sample = self.source[idx]
        target_sample = self.target[idx]

        return source_sample, target_sample

    def __len__(self):
        return len(self.source)
    

In [ ]:
yourDataset = Dataset()
dataloader = torch.utils.data.DataLoader(
        yourDataset,
        batch_size=8,
        num_workers=0,
        shuffle=True
)


In [ ]:
print(dataloader.)

In [1]:
with open("./training_data/Roman-Urdu.txt", "r", encoding="utf-8") as f:
    roman_urdu_data = f.read()

with open("./training_data/Urdu.txt", "r", encoding="utf-8") as f:
    urdu_data = f.read()



In [2]:
print(urdu_data.splitlines())

['وکیل ', 'وکیل    ', 'وکیل     ایک ایسی شخصیت کو کہا جاتا ہے کہ جو دوسرے  اپنے صارف  کی جانب سے یا اسکی بابت گفتگو کرے   اس مضمون میں یہ گفتگو قانون سے متعلق تصور کی گئی ہے اور اس وجہ سے یہ مضمون صرف قانونی وکلا  کے بارے میں ذکر کرتا ہے', 'وکیل  قانون ', 'وکیل  قانون  ایک شخص جسے دوسرے شخص کی جگہ کام کرنے یا     کی نمائندگی کرنے کا اختیار حاصل ہوتا ہے  وکیل  سفر  ایک شخص جو تعطیلات اور سفر کا بندوبست کرتا ہے  خفیہ وکیل  ایک جاسوس ', 'وکیل بازار ضلع   لاطینی       ترکمانستان کا ایک ترکمانستان کے اضلاع جو صوبہ ماری میں واقع ہے', 'وکیل والا ریلوے اسٹیشن وکیل والا میں واقع ہے', 'وکیل والا ریلوے اسٹیشن پاکستان میں واقع ہے', 'وکیپیڈیا', 'وکیپیڈیا انگریزی پر یہ تصویر ميں نے ہی اپلوڈ کی تھی', 'وکیپیڈیا خود کسی بھی ترتیب کو نافذ نہیں کرتا', 'وکیپیڈیا لوگو کے لیے نستعلیق خطاطی', 'وکیپیڈیا میں اس سطر سے آپ تصاویر سادہ طریقے سے شامل کرسکتے ہیں ', 'وکیپیڈیا میں خوش آمدید', 'وکیپیڈیا میں کسی کے کو اس بات کی وجہ سے فوقیت حاصل نہیں کہ اس نے کس قدر وکیپیڈیا میں اضافہ کیا ہے', 'وکیپیڈیا پالیسی ', 'وکیپ

In [3]:
roman_urdu_lines = roman_urdu_data.splitlines()
urdu_lines = urdu_data.splitlines()

In [4]:
print(roman_urdu_lines, urdu_lines[3].split())

['wakeel', 'wakeel', 'wakeel aik aisi shakhsiyat ko kaha jata hai ke jo dosray apne Sarif ki janib se ya uski baabat guftagu kere is mazmoon mein yeh guftagu qanoon se mutaliq tasawwur ki gayi hai aur is wajah se yeh mazmoon sirf qanooni wukla ke baray mein zikar karta hai', 'wakeel qanoon', 'wakeel qanoon 1 shakhs jisay dosray shakhs ki jagah kaam karne ya ki numaindagi karne ka ikhtiyar haasil hota hai wakeel safar 1 shakhs jo tatilat aur safar ka bandobast karta hai khufia wakeel aik jasoos', 'wakeel bazaar zila lateeni turkmenistan ka aik turkmenistan ke azlaa jo soobah maari mein waqay hai', 'wakeel wala railway station wakeel wala mein waqay hai', 'wakeel wala railway station Pakistan mein waqay hai', 'wikipedia', 'wikipedia angrezi par yeh tasweer mein ne hi aplod ki thi', 'wikipedia khud kisi bhi tarteeb ko nafiz nahi karta', 'wikipedia logo ke liye nastalik khtati', 'wikipedia mein is satar se aap tasaveer saada tareeqay se shaamil kar saktay hain', 'wikipedia mein khush aamde

In [5]:
def getUniqueWords(lines):

    vocab = []
    for i in lines:
        toks = i.split()

        for j in toks:
            if j not in vocab:
                vocab.append(j)

    return vocab

Get Vocab of roman urdu and urdu

In [6]:
import torchtext.vocab as vocab

vocab_roman = getUniqueWords(roman_urdu_lines)
vocab_urdu = getUniqueWords(urdu_lines)

Urdu Vocab only first 35 words

In [28]:
print(len(vocab_urdu))
vocab_urdu[:35]

337


['وکیل',
 'ایک',
 'ایسی',
 'شخصیت',
 'کو',
 'کہا',
 'جاتا',
 'ہے',
 'کہ',
 'جو',
 'دوسرے',
 'اپنے',
 'صارف',
 'کی',
 'جانب',
 'سے',
 'یا',
 'اسکی',
 'بابت',
 'گفتگو',
 'کرے',
 'اس',
 'مضمون',
 'میں',
 'یہ',
 'قانون',
 'متعلق',
 'تصور',
 'گئی',
 'اور',
 'وجہ',
 'صرف',
 'قانونی',
 'وکلا',
 'کے']

In [27]:
print(len(vocab_roman))
vocab_roman[:35]

339


['wakeel',
 'aik',
 'aisi',
 'shakhsiyat',
 'ko',
 'kaha',
 'jata',
 'hai',
 'ke',
 'jo',
 'dosray',
 'apne',
 'Sarif',
 'ki',
 'janib',
 'se',
 'ya',
 'uski',
 'baabat',
 'guftagu',
 'kere',
 'is',
 'mazmoon',
 'mein',
 'yeh',
 'qanoon',
 'mutaliq',
 'tasawwur',
 'gayi',
 'aur',
 'wajah',
 'sirf',
 'qanooni',
 'wukla',
 'baray']

In [15]:
f = list(zip(vocab_roman, vocab_urdu))

In [21]:
# Create a dictionary that maps each word to an index
roman_word_to_ix = {word: i for i, word in enumerate(vocab_roman)}
urdu_word_to_ix = {word: i for i, word in enumerate(vocab_urdu)}

print(roman_word_to_ix)
print(urdu_word_to_ix)


{'wakeel': 0, 'aik': 1, 'aisi': 2, 'shakhsiyat': 3, 'ko': 4, 'kaha': 5, 'jata': 6, 'hai': 7, 'ke': 8, 'jo': 9, 'dosray': 10, 'apne': 11, 'Sarif': 12, 'ki': 13, 'janib': 14, 'se': 15, 'ya': 16, 'uski': 17, 'baabat': 18, 'guftagu': 19, 'kere': 20, 'is': 21, 'mazmoon': 22, 'mein': 23, 'yeh': 24, 'qanoon': 25, 'mutaliq': 26, 'tasawwur': 27, 'gayi': 28, 'aur': 29, 'wajah': 30, 'sirf': 31, 'qanooni': 32, 'wukla': 33, 'baray': 34, 'zikar': 35, 'karta': 36, '1': 37, 'shakhs': 38, 'jisay': 39, 'jagah': 40, 'kaam': 41, 'karne': 42, 'numaindagi': 43, 'ka': 44, 'ikhtiyar': 45, 'haasil': 46, 'hota': 47, 'safar': 48, 'tatilat': 49, 'bandobast': 50, 'khufia': 51, 'jasoos': 52, 'bazaar': 53, 'zila': 54, 'lateeni': 55, 'turkmenistan': 56, 'azlaa': 57, 'soobah': 58, 'maari': 59, 'waqay': 60, 'wala': 61, 'railway': 62, 'station': 63, 'Pakistan': 64, 'wikipedia': 65, 'angrezi': 66, 'par': 67, 'tasweer': 68, 'ne': 69, 'hi': 70, 'aplod': 71, 'thi': 72, 'khud': 73, 'kisi': 74, 'bhi': 75, 'tarteeb': 76, 'nafi

KeyError: 0

In [24]:
# Convert To indeces

urdu_word_indices = torch.tensor([urdu_word_to_ix[word] for word in vocab_urdu], dtype=torch.long)


roman_word_indices = torch.tensor([roman_word_to_ix[word] for word in vocab_roman], dtype=torch.long)



tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [25]:
print(urdu_word_indices)

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [26]:
print(roman_word_indices)


tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [31]:
import torch
import torch.nn as nn

embedding = nn.Embedding(num_embeddings=500, embedding_dim=10)


roman_embedded_words = embedding(roman_word_indices)
urdu_embedded_words = embedding(urdu_word_indices)

In [32]:
print(urdu_embedded_words)

tensor([[-0.4767, -0.3058,  0.2344,  ...,  0.5078, -0.9380, -1.0690],
        [ 1.2438, -0.1743, -1.7857,  ...,  2.1216,  0.6853,  0.8915],
        [ 0.8893,  1.5156,  1.4799,  ..., -0.0074, -0.2525,  2.0934],
        ...,
        [ 0.2276,  0.1307, -1.5816,  ...,  0.8715,  0.0811,  0.3351],
        [-0.0235, -0.9776,  1.6524,  ...,  0.1228, -0.2487, -1.1017],
        [-0.7499,  1.1364,  0.6656,  ..., -0.4363,  0.9443, -1.5120]],
       grad_fn=<EmbeddingBackward0>)


In [16]:
import torch.nn as nn
import torch

embedding = nn.Embedding(num_embeddings=10000, embedding_dim=128)
embeddings = embedding(torch.LongTensor(f))


ValueError: too many dimensions 'str'

In [72]:
glove_urdu = vocab.GloVe(name='6B', dim=100)

BadZipFile: File is not a zip file

In [27]:
import re
import random
import torch
import torchtext
from torch.utils.data import TensorDataset, DataLoader

# Read the contents of both files and store them in separate variables
with open("./training_data/Roman-Urdu.txt", "r", encoding="utf-8") as f:
    roman_urdu_data = f.read()

with open("./training_data/Urdu.txt", "r", encoding="utf-8") as f:
    urdu_data = f.read()

# Split the data in both files into sentences
roman_urdu_sentences = re.split(r"[.\n]", roman_urdu_data)
urdu_sentences = re.split(r"[.\n]", urdu_data)

# Remove any unnecessary characters and tokenize the sentences
roman_urdu_sentences = [re.sub(r"[^a-zA-Z0-9آ-ی]+", " ", s).strip().split()[:10] for s in roman_urdu_sentences]
urdu_sentences = [re.sub(r"[^آ-ی۔]+", " ", s).strip().split()[:10] for s in urdu_sentences]

# Create separate lists for Urdu and Roman Urdu sentences
data = list(zip(roman_urdu_sentences, urdu_sentences))
random.shuffle(data)
roman_urdu_sentences, urdu_sentences = zip(*data)



In [28]:
print(roman_urdu_sentences[0][0], urdu_sentences[0][0])
print('gg')

woh وہ
gg


In [35]:
print(data)
roman_urdu_sentences.size()

[(['woh', 'ke', 'awail', 'mein', 'himayat', 'Ali', 'Karachi', 'aakar', 'radio', 'Pakistan'], ['وہ', 'ک', 'اوائل', 'میں', 'حمایت', 'علی', 'کراچی', 'آکر', 'ریڈیو', 'پاکستان']), (['woh', 'mein', 'ensaan', 'ki', 'qowat', 'mudafat', 'ko', 'kamzor', 'kar', 'dainay'], ['وہ', 'میں', 'انسان', 'کی', 'قوت', 'مدافعت', 'کو', 'کمزور', 'کر', 'دین']), (['woh', 'se', 'le', 'kar', 'taq', 'hukoomat', 'Pakistan', 'ke', 'wakeel', 'rahay'], ['وہ', 'س', 'ل', 'کر', 'تک', 'حکومت', 'پاکستان', 'ک', 'وکیل', 'رہ']), (['wikipedia', 'par', 'sirf', 'hazaar', 'mazmoon', 'hain', 'jin', 'mein', 'se', 'asal'], ['وکیپیڈیا', 'پر', 'صرف', 'ہزار', 'مضمون', 'ہیں', 'جن', 'میں', 'س', 'اصل']), (['woh', 'se', 'taq', 'Bharat', 'ke', 'Wazeer', 'e', 'Azam', 'rahay'], ['وہ', 'س', 'تک', 'بھارت', 'ک', 'وزیر', 'اعظم', 'رہ']), (['wakeel', 'wala', 'railway', 'station', 'Pakistan', 'mein', 'waqay', 'hai'], ['وکیل', 'والا', 'ریلو', 'اسٹیشن', 'پاکستان', 'میں', 'واقع', 'ہ']), (['woh', 'mein', 'markazi', 'urdu', 'board', 'ke', 'director', 'muq

AttributeError: 'tuple' object has no attribute 'size'

In [37]:
# Split the data into training and validation sets
split_idx = int(0.8 * len(data))
train_data = data[:split_idx]
valid_data = data[split_idx:]

# Convert the sentences to PyTorch tensors
def to_tensor(sentence):
    
    token_ids = []
    for word in sentence:
        if word in vocab:
            token_ids.append(vocab[word])
        else:
            token_ids.append(vocab['<unk>'])
    return torch.tensor(token_ids, dtype=torch.long)

vocab = torchtext.vocab.build_vocab_from_iterator(
    [word for sentence in roman_urdu_sentences + urdu_sentences for word in sentence],
    specials=["<pad>", "<unk>"]
)

train_roman_urdu = [to_tensor(s) for s, _ in train_data if len(s) > 0]
train_urdu = [to_tensor(s) for _, s in train_data if len(s) > 0]
valid_roman_urdu = [to_tensor(s) for s, _ in valid_data if len(s) > 0]
valid_urdu = [to_tensor(s) for _, s in valid_data if len(s) > 0]

print(train_roman_urdu.size())



AttributeError: 'list' object has no attribute 'size'

In [ ]:
train_dataset = TensorDataset(*train_roman_urdu, *train_urdu)
valid_dataset = TensorDataset(valid_roman_urdu, valid_urdu)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)